In [1]:
from catboost import CatBoostClassifier
from imblearn.over_sampling import ADASYN, SMOTE, SMOTENC
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, r2_score, roc_auc_score
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

/home/topshik/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/topshik/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/topshik/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/topshik/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

In [2]:
#!wget https://www.openml.org/data/get_csv/53995/KDDCup09_churn.csv

In [3]:
!ls

catboost_info
cat_feature_index.281b0f26-bb6997e9-c1dde8e5-a478fdf9.tmp
cat_feature_index.3510457a-941c9e1c-57753b69-5eb7ac48.tmp
cat_feature_index.38cb27cd-cc63e158-fbe7d73f-e423e683.tmp
cat_feature_index.581296e9-30462c5d-af64deea-35ad8944.tmp
cat_feature_index.7ebc92d3-223f27a9-b897d74b-69fa660c.tmp
cat_feature_index.b3967a73-d6d526a4-3767ffb2-6bcfc834.tmp
cat_feature_index.dad7d190-20a7ddf1-900642d-603e679b.tmp
cat_feature_index.eae79a34-860c607f-67a32e5f-faa56d99.tmp
cat_feature_index.ed7642d3-5af8a134-cdda4a56-491dbffe.tmp
cat_feature_index.f72c2ac4-5e3d457e-2e4a946e-ec82b76.tmp
classification_filled.ipynb
hw1
hw2
KDDCup09_churn.csv
model_complexity_filled.ipynb
proj.ipynb
regression.ipynb


In [4]:
pd.set_option('display.max_columns', 240)
data = pd.read_csv('KDDCup09_churn.csv')
print(data.shape)
data.head(15)

(50000, 231)


,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,Var11,Var12,Var13,Var14,Var15,Var16,Var17,Var18,Var19,Var20,Var21,Var22,Var23,Var24,Var25,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33,Var34,Var35,Var36,Var37,Var38,Var39,Var40,Var41,Var42,Var43,Var44,Var45,Var46,Var47,Var48,Var49,Var50,Var51,Var52,Var53,Var54,Var55,Var56,Var57,Var58,Var59,Var60,Var61,Var62,Var63,Var64,Var65,Var66,Var67,Var68,Var69,Var70,Var71,Var72,Var73,Var74,Var75,Var76,Var77,Var78,Var79,Var80,Var81,Var82,Var83,Var84,Var85,Var86,Var87,Var88,Var89,Var90,Var91,Var92,Var93,Var94,Var95,Var96,Var97,Var98,Var99,Var100,Var101,Var102,Var103,Var104,Var105,Var106,Var107,Var108,Var109,Var110,Var111,Var112,Var113,Var114,Var115,Var116,Var117,Var118,Var119,Var120,Var121,Var122,Var123,Var124,Var125,Var126,Var127,Var128,Var129,Var130,Var131,Var132,Var133,Var134,Var135,Var136,Var137,Var138,Var139,Var140,Var141,Var142,Var143,Var144,Var145,Var146,Var147,Var148,Var149,Var150,Var151,Var152,Var153,Var154,Var155,Var156,Var157,Var158,Var159,Var160,Var161,Var162,Var163,Var164,Var165,Var166,Var167,Var168,Var169,Var170,Var171,Var172,Var173,Var174,Var175,Var176,Var177,Var178,Var179,Var180,Var181,Var182,Var183,Var184,Var185,Var186,Var187,Var188,Var189,Var190,Var191,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,Var201,Var202,Var203,Var204,Var205,Var206,Var207,Var208,Var209,Var210,Var211,Var212,Var213,Var214,Var215,Var216,Var217,Var218,Var219,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,CHURN
0,?,?,?,?,?,1526,7,?,?,?,?,?,184,?,?,?,?,?,?,?,464,580,?,14,128,?,?,166.56,?,?,?,?,?,?,0,?,?,3570,?,?,?,?,?,0,?,?,?,?,?,?,?,?,?,?,?,?,4.076907,?,?,?,?,?,?,?,9,?,?,?,?,?,?,?,36,35,?,1350864,?,0,?,?,7333.11,?,5,?,12,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,104,?,?,168,117625.60,?,?,?,?,?,1175,?,?,?,6,?,720,8,?,?,?,?,?,0,1212385,69134,?,?,?,?,?,185,?,?,0,9,?,?,?,?,397579,?,?,?,1812252,?,?,?,?,?,?,142,?,?,38418,?,?,?,?,?,?,?,?,?,0,?,?,?,?,?,?,?,0,?,?,?,?,?,?,?,462,?,?,bZkvyxLkBI,RO12,?,taul,1K8T,lK27,ka_ns41,nQUveAzAF7,?,?,dXGu,9_Y1,FbIm,VpdQ,haYg,me75fM6ugJ,kIsH,?,uKAI,L84s,XfqtO3UdzaXh_,?,?,?,XTbPUYD,sH5Z,cJvF,FzaX,1YVfGrO,oslk,fXVEsaq,jySVZNlOJy,?,?,xb3V,RAYp,F2FyR07IdsN7I,?,?,-1
1,?,?,?,?,?,525,0,?,?,?,?,?,0,?,?,?,?,?,?,?,168,210,?,2,24,?,?,353.52,?,?,?,?,?,?,0,?,?,4764966,?,?,?,?,?,0,?,?,?,?,?,?,?,?,?,?,?,?,5.408032,?,?,?,?,?,?,?,9,?,?,?,?,?,?,3,26,0,?,2872928,?,3,?,?,151098.9,?,25,?,2,?,?,?,?,?,?,?,?,58158,?,?,?,?,?,?,?,?,?,?,?,?,?,?,40,?,?,40,-356411.60,?,?,?,?,?,590,?,?,?,72,?,0,?,?,?,?,?,?,8,4136430,357038,?,?,?,?,?,0,?,?,0,9,?,?,?,?,278334,?,?,?,10439160,?,?,?,?,?,?,32,?,?,238572,?,?,?,?,?,?,?,?,?,0,?,?,?,?,?,?,?,0,?,?,?,?,?,?,?,?,?,?,CEat0G8rTN,RO12,?,taul,1K8T,2Ix5,qEdASpP,y2LIM01bE1,?,?,lg1t,9_Y1,k13i,sJzTlal,zm5i,me75fM6ugJ,kIsH,?,uKAI,L84s,NhsEn4L,?,?,?,kZJyVg2,?,?,FzaX,0AJo2f2,oslk,2Kb5FSF,LM8l689qOp,?,?,fKCe,RAYp,F2FyR07IdsN7I,?,?,1
2,?,?,?,?,?,5236,7,?,?,?,?,?,904,?,?,?,?,?,?,?,1212,1515,?,26,816,?,?,220.08,?,?,?,?,?,?,0,?,?,5883894,?,?,?,?,?,0,?,?,?,?,?,?,?,?,?,?,?,?,6.599658,?,?,?,?,?,?,?,9,?,?,?,?,?,?,?,130,518,?,1675776,?,0,?,?,16211.58,?,40,?,58,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,312,?,?,336,405104.00,?,?,?,?,?,3230,?,?,?,114,?,5967,-28,?,?,?,?,?,0,3478905,248932,?,?,?,?,?,800,?,?,0,36,?,?,?,?,320565,?,?,?,9826360,?,?,?,?,?,?,206,?,?,434946,?,?,?,?,?,?,?,?,?,0,?,?,?,?,?,?,?,0,?,?,?,?,?,?,?,?,?,?,eOQt0GoOh3,AERks4l,SEuy,taul,1K8T,ffXs,NldASpP,y4g9XoZ,vynJTq9,smXZ,4bTR,9_Y1,MGOA,VpdQ,haYg,DHn_WUyBhW_whjA88g9bvA64_,kIsH,?,uKAI,L84s,UbxQ8lZ,?,TTGHfSv,?,pMWAe2U,bHR7,UYBR,FzaX,JFM1BiF,Al6ZaUT,NKv4yOc,jySVZNlOJy,?,kG3k,Qu4f,02N6s8f,ib5G6X1eUxUn6,am7c,?,-1
3,?,?,?,?,?,?,0,?,?,?,?,?,0,?,?,?,?,?,?,?,?,0,?,?,0,?,?,22.08,?,?,?,?,?,?,0,?,?,0,?,?,?,?,?,0,?,?,?,?,?,?,?,?,?,?,?,?,1.988250,?,?,?,?,?,?,?,9,?,?,?,?,?,?,?,12,0,?,0,?,0,?,?,?,?,0,?,0,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0,-275703.60,?,?,?,?,?,?,?,?,?,0,?,0,-14,?,?,?,?,?,0,0,0,?,?,?,?,?,0,?,?,0,?,?,?,?,?,?,?,?,?,0,?,?,?,?,?,?,0,?,?,0,?,?,?,?,?,?,?,?,?,0,

In [5]:
print('\n', data.dtypes.value_counts())
print('\nTarget type: ', data.CHURN.dtypes)


 object     227
float64      2
int64        2
dtype: int64

Target type:  int64


# Catboost model

In [6]:
def preprocess(data):
    num_columns = []
    cat_columns = []

    print('Shape before: ', data.shape)
    
    # delete empty and almost empty columns
    for column in data.drop(columns='CHURN').columns:
        is_empty = data[column].unique().shape[0] == 1 and data[column].unique()[0] == '?'
        helper = data[column].value_counts()
        is_almost_empty = '?' in helper.index.tolist()
        if is_empty or (is_almost_empty and helper['?'] > 20000):
            data = data.drop(columns=column)
        else:
            if int(column[3:]) <= 190:
                num_columns.append(column)
            else:
                cat_columns.append(column)
    
    print('Shape after: ', data.shape)
    
    # make replacements in numerical and categorical columns
    for column in num_columns:
        data[column] = data[column].replace('?', np.nan)
        data[column] = data[column].astype('float64')
        data[column] = data[column].fillna(data[column].mean())
    
    for column in cat_columns:
        data[column] = data[column].replace(np.nan, 'kek')
        data[column] = data[column].astype('str')
    
    # perform ordinal encoding for categorical features
    X = data.drop(columns='CHURN')
    enc = OrdinalEncoder()
    X[cat_columns] = enc.fit_transform(X[cat_columns])
    X = pd.DataFrame(X)
    
    
    # indices of categorical columns
    cat_columns_idx = []
    for column in cat_columns:
        cat_columns_idx.append(data.columns.get_loc(column))
        
    data.CHURN = data.CHURN.replace(-1, 0)

    return (data, X, num_columns, cat_columns, cat_columns_idx)


def evaluate_model(model, X_test, y_test):
    preds = model.predict(X_test)
    preds_proba = model.predict_proba(X_test)
    print('Accuracy: ', accuracy_score(y_test, preds))
    print('Roc auc: ', roc_auc_score(y_test, preds_proba[:, 1]))
    print('f1 score: ', f1_score(y_test, preds))

In [7]:
data, X, num_columns, cat_columns, cat_columns_idx = preprocess(data)

Shape before:  (50000, 231)
Shape after:  (50000, 68)


In [8]:
clf = CatBoostClassifier(cat_features=cat_columns_idx)
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns='CHURN'), data.CHURN)
X_resampled, y_resampled = SMOTENC(categorical_features=cat_columns_idx, sampling_strategy=0.33).fit_resample(X_train, y_train)

In [9]:
clf.fit(X_resampled, y_resampled)

Learning rate set to 0.049572
0:	learn: 0.6522551	total: 120ms	remaining: 1m 59s
1:	learn: 0.6188706	total: 186ms	remaining: 1m 32s
2:	learn: 0.5915365	total: 267ms	remaining: 1m 28s
3:	learn: 0.5666824	total: 330ms	remaining: 1m 22s
4:	learn: 0.5311485	total: 425ms	remaining: 1m 24s
5:	learn: 0.5048725	total: 498ms	remaining: 1m 22s
6:	learn: 0.4752632	total: 584ms	remaining: 1m 22s
7:	learn: 0.4506235	total: 675ms	remaining: 1m 23s
8:	learn: 0.4335497	total: 745ms	remaining: 1m 22s
9:	learn: 0.4132865	total: 831ms	remaining: 1m 22s
10:	learn: 0.3941933	total: 934ms	remaining: 1m 23s
11:	learn: 0.3744467	total: 1.04s	remaining: 1m 25s
12:	learn: 0.3583879	total: 1.15s	remaining: 1m 27s
13:	learn: 0.3459717	total: 1.26s	remaining: 1m 28s
14:	learn: 0.3343656	total: 1.36s	remaining: 1m 29s
15:	learn: 0.3231708	total: 1.5s	remaining: 1m 32s
16:	learn: 0.3130864	total: 1.58s	remaining: 1m 31s
17:	learn: 0.3025524	total: 1.69s	remaining: 1m 32s
18:	learn: 0.2929982	total: 1.8s	remaining: 1

158:	learn: 0.1030905	total: 20.5s	remaining: 1m 48s
159:	learn: 0.1025310	total: 20.6s	remaining: 1m 48s
160:	learn: 0.1021873	total: 20.8s	remaining: 1m 48s
161:	learn: 0.1019764	total: 20.9s	remaining: 1m 48s
162:	learn: 0.1016719	total: 21.1s	remaining: 1m 48s
163:	learn: 0.1014505	total: 21.3s	remaining: 1m 48s
164:	learn: 0.1010158	total: 21.5s	remaining: 1m 48s
165:	learn: 0.1007502	total: 21.6s	remaining: 1m 48s
166:	learn: 0.1005692	total: 21.7s	remaining: 1m 48s
167:	learn: 0.1003927	total: 21.9s	remaining: 1m 48s
168:	learn: 0.1002952	total: 22s	remaining: 1m 48s
169:	learn: 0.1000633	total: 22.1s	remaining: 1m 48s
170:	learn: 0.0999212	total: 22.3s	remaining: 1m 47s
171:	learn: 0.0998527	total: 22.3s	remaining: 1m 47s
172:	learn: 0.0998441	total: 22.4s	remaining: 1m 47s
173:	learn: 0.0997945	total: 22.5s	remaining: 1m 46s
174:	learn: 0.0994882	total: 22.6s	remaining: 1m 46s
175:	learn: 0.0994747	total: 22.6s	remaining: 1m 46s
176:	learn: 0.0993605	total: 22.7s	remaining: 1m

316:	learn: 0.0831522	total: 38.8s	remaining: 1m 23s
317:	learn: 0.0830245	total: 38.9s	remaining: 1m 23s
318:	learn: 0.0830177	total: 39s	remaining: 1m 23s
319:	learn: 0.0829690	total: 39s	remaining: 1m 22s
320:	learn: 0.0829486	total: 39.1s	remaining: 1m 22s
321:	learn: 0.0828940	total: 39.3s	remaining: 1m 22s
322:	learn: 0.0828367	total: 39.4s	remaining: 1m 22s
323:	learn: 0.0826238	total: 39.5s	remaining: 1m 22s
324:	learn: 0.0826215	total: 39.6s	remaining: 1m 22s
325:	learn: 0.0825816	total: 39.7s	remaining: 1m 22s
326:	learn: 0.0825071	total: 39.9s	remaining: 1m 22s
327:	learn: 0.0823649	total: 40s	remaining: 1m 22s
328:	learn: 0.0823576	total: 40.1s	remaining: 1m 21s
329:	learn: 0.0823045	total: 40.2s	remaining: 1m 21s
330:	learn: 0.0821906	total: 40.3s	remaining: 1m 21s
331:	learn: 0.0821586	total: 40.4s	remaining: 1m 21s
332:	learn: 0.0821458	total: 40.5s	remaining: 1m 21s
333:	learn: 0.0821394	total: 40.6s	remaining: 1m 20s
334:	learn: 0.0821332	total: 40.6s	remaining: 1m 20s

473:	learn: 0.0756163	total: 54s	remaining: 59.9s
474:	learn: 0.0756135	total: 54.1s	remaining: 59.7s
475:	learn: 0.0756023	total: 54.1s	remaining: 59.6s
476:	learn: 0.0755774	total: 54.2s	remaining: 59.4s
477:	learn: 0.0755644	total: 54.3s	remaining: 59.3s
478:	learn: 0.0755100	total: 54.4s	remaining: 59.2s
479:	learn: 0.0754722	total: 54.5s	remaining: 59.1s
480:	learn: 0.0754710	total: 54.6s	remaining: 58.9s
481:	learn: 0.0754594	total: 54.7s	remaining: 58.8s
482:	learn: 0.0754494	total: 54.7s	remaining: 58.6s
483:	learn: 0.0754388	total: 54.8s	remaining: 58.4s
484:	learn: 0.0753539	total: 54.9s	remaining: 58.3s
485:	learn: 0.0753352	total: 55s	remaining: 58.2s
486:	learn: 0.0753339	total: 55s	remaining: 58s
487:	learn: 0.0753325	total: 55.1s	remaining: 57.8s
488:	learn: 0.0753132	total: 55.2s	remaining: 57.7s
489:	learn: 0.0752864	total: 55.3s	remaining: 57.5s
490:	learn: 0.0752831	total: 55.3s	remaining: 57.4s
491:	learn: 0.0752599	total: 55.4s	remaining: 57.2s
492:	learn: 0.075255

632:	learn: 0.0701343	total: 1m 9s	remaining: 40.1s
633:	learn: 0.0700458	total: 1m 9s	remaining: 39.9s
634:	learn: 0.0700402	total: 1m 9s	remaining: 39.8s
635:	learn: 0.0699588	total: 1m 9s	remaining: 39.7s
636:	learn: 0.0699563	total: 1m 9s	remaining: 39.6s
637:	learn: 0.0699544	total: 1m 9s	remaining: 39.5s
638:	learn: 0.0699322	total: 1m 9s	remaining: 39.4s
639:	learn: 0.0699296	total: 1m 9s	remaining: 39.2s
640:	learn: 0.0699120	total: 1m 9s	remaining: 39.1s
641:	learn: 0.0697684	total: 1m 10s	remaining: 39s
642:	learn: 0.0697504	total: 1m 10s	remaining: 38.9s
643:	learn: 0.0697495	total: 1m 10s	remaining: 38.8s
644:	learn: 0.0696995	total: 1m 10s	remaining: 38.7s
645:	learn: 0.0696882	total: 1m 10s	remaining: 38.5s
646:	learn: 0.0696407	total: 1m 10s	remaining: 38.4s
647:	learn: 0.0695958	total: 1m 10s	remaining: 38.3s
648:	learn: 0.0694370	total: 1m 10s	remaining: 38.2s
649:	learn: 0.0694211	total: 1m 10s	remaining: 38.1s
650:	learn: 0.0693260	total: 1m 10s	remaining: 38s
651:	l

788:	learn: 0.0650360	total: 1m 25s	remaining: 23s
789:	learn: 0.0650336	total: 1m 25s	remaining: 22.8s
790:	learn: 0.0650320	total: 1m 25s	remaining: 22.7s
791:	learn: 0.0650216	total: 1m 26s	remaining: 22.6s
792:	learn: 0.0650146	total: 1m 26s	remaining: 22.5s
793:	learn: 0.0649955	total: 1m 26s	remaining: 22.4s
794:	learn: 0.0649860	total: 1m 26s	remaining: 22.3s
795:	learn: 0.0649415	total: 1m 26s	remaining: 22.1s
796:	learn: 0.0649412	total: 1m 26s	remaining: 22s
797:	learn: 0.0649243	total: 1m 26s	remaining: 21.9s
798:	learn: 0.0649030	total: 1m 26s	remaining: 21.8s
799:	learn: 0.0648975	total: 1m 26s	remaining: 21.7s
800:	learn: 0.0648928	total: 1m 26s	remaining: 21.6s
801:	learn: 0.0648732	total: 1m 27s	remaining: 21.5s
802:	learn: 0.0648652	total: 1m 27s	remaining: 21.4s
803:	learn: 0.0648356	total: 1m 27s	remaining: 21.3s
804:	learn: 0.0648344	total: 1m 27s	remaining: 21.2s
805:	learn: 0.0648214	total: 1m 27s	remaining: 21s
806:	learn: 0.0648124	total: 1m 27s	remaining: 20.9s

945:	learn: 0.0613628	total: 1m 40s	remaining: 5.75s
946:	learn: 0.0613608	total: 1m 40s	remaining: 5.64s
947:	learn: 0.0613007	total: 1m 40s	remaining: 5.53s
948:	learn: 0.0612501	total: 1m 40s	remaining: 5.43s
949:	learn: 0.0612456	total: 1m 41s	remaining: 5.32s
950:	learn: 0.0611337	total: 1m 41s	remaining: 5.22s
951:	learn: 0.0610551	total: 1m 41s	remaining: 5.11s
952:	learn: 0.0610331	total: 1m 41s	remaining: 5s
953:	learn: 0.0610320	total: 1m 41s	remaining: 4.9s
954:	learn: 0.0610164	total: 1m 41s	remaining: 4.79s
955:	learn: 0.0610063	total: 1m 41s	remaining: 4.69s
956:	learn: 0.0609938	total: 1m 41s	remaining: 4.58s
957:	learn: 0.0609888	total: 1m 41s	remaining: 4.47s
958:	learn: 0.0609800	total: 1m 42s	remaining: 4.36s
959:	learn: 0.0609742	total: 1m 42s	remaining: 4.26s
960:	learn: 0.0609597	total: 1m 42s	remaining: 4.15s
961:	learn: 0.0609444	total: 1m 42s	remaining: 4.04s
962:	learn: 0.0609280	total: 1m 42s	remaining: 3.94s
963:	learn: 0.0608968	total: 1m 42s	remaining: 3.8

In [10]:
evaluate_model(clf, X_test, y_test)

Accuracy:  0.92544
Roc auc:  0.687829136508251
f1 score:  0.018947368421052633


### Grid search catboost

In [12]:
cat_clf_cv = CatBoostClassifier(cat_features=cat_columns_idx)
param_grid = {
    'learning_rate': np.logspace(-4, -1, 4),
    'depth': np.linspace(10, 16, 4).astype('int64'),
    'eval_metric': ['Logloss', 'AUC'],
}
grid = GridSearchCV(cat_clf_cv, param_grid=param_grid, scoring='f1', n_jobs=-1, cv=4, verbose=True)
grid.fit(X_resampled, y_resampled)

Fitting 4 folds for each of 32 candidates, totalling 128 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


KeyboardInterrupt: 

In [ ]:
evaluate_model(cat_clf_cv, X_test, y_test)

# Logistic regression

In [230]:
scaler = StandardScaler()
scaler.fit(X[num_columns])
X[num_columns] = scaler.transform(X[num_columns])

In [235]:
log_reg_clf = LogisticRegression(solver='lbfgs')
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(X, data.CHURN)
X_resampled, y_resampled = SMOTENC(categorical_features=cat_columns_idx, sampling_strategy=0.33).fit_resample(X_train_log, y_train_log)

In [236]:
log_reg_clf.fit(X_resampled, y_resampled)
log_reg_clf.predict(X_test_log)

evaluate_model(log_reg_clf, X_test_log, y_test_log)

Accuracy:  0.9252
Roc auc:  0.550985339779114
f1 score:  0.0


/home/topshik/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/topshik/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
